In [ ]:
#!pip install --upgrade pip
#!pip install --upgrade setuptools
#!pip install GEOS
#!pip install GeobricksProj4ToEPSG
#!conda install -c conda-forge cartopy --yes
#!pip install geopandas
print('ok')

In [ ]:
import os
import pandas
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import ticker
from IPython.display import display, HTML
import geopandas as gpd
import time
from shapely.geometry import Polygon

os.chdir('/home/idies/workspace/Temporary/raddick/cra_scratch/')
print(os.getcwd())

In [ ]:
shapefile_dir = '/home/idies/workspace/Storage/raddick/persistent/baltimore_neighborhoods/shapefiles/'

s = time.time()
nfilename = shapefile_dir + 'Neighborhoods.shp'
#nfile = '/home/idies/workspace/Storage/raddick/persistent/baltimore_neighborhoods/shapefiles/Neighborhoods.shp'
neighborhood_gdf = gpd.read_file(nfilename)
neighborhood_gdf = neighborhood_gdf.set_index('Name')
e = time.time()
print('Read {0:,.0f} neighborhoods in {1:.3f} seconds.'.format(len(neighborhood_gdf), e-s))

s = time.time()
tracts_2010_filename = shapefile_dir + 'census_tracts_2010/geo_export_c50bbe56-543e-4878-9c9f-c56be327600a.shp'
tracts_2010_gdf = gpd.read_file(tracts_2010_filename, encoding='utf-8')
tracts_2010_gdf = tracts_2010_gdf.assign(tractname=pandas.to_numeric(tracts_2010_gdf['name'].apply(lambda x: x.split(' ')[-1]), errors='coerce'))
tracts_2010_gdf = tracts_2010_gdf.set_index('tractname')
tracts_2010_gdf = tracts_2010_gdf.to_crs(neighborhood_gdf.crs)
e = time.time()
print('Got {0:,.0f} 2010 census tracts in {1:.3f} seconds.'.format(len(tracts_2010_gdf), e-s))

s = time.time()
tracts_2017_filename = shapefile_dir + 'tl_2017_24_tract/tl_2017_24_tract.shp'
tracts_2017_gdf = gpd.read_file(tracts_2017_filename, encoding='utf-8')
tracts_2017_gdf['STATEFP'] = pandas.to_numeric(tracts_2017_gdf['STATEFP'], errors='coerce')
tracts_2017_gdf['COUNTYFP'] = pandas.to_numeric(tracts_2017_gdf['COUNTYFP'], errors='coerce')
tracts_2017_gdf = tracts_2017_gdf[(tracts_2017_gdf['STATEFP'] == 24) & (tracts_2017_gdf['COUNTYFP'] == 510)]
tracts_2017_gdf = tracts_2017_gdf.assign(tractname = pandas.to_numeric(tracts_2017_gdf['NAME'], errors='coerce'))
tracts_2017_gdf = tracts_2017_gdf.set_index('tractname')
tracts_2017_gdf = tracts_2017_gdf.join(tracts_2010_gdf['population'], how='left')
tracts_2017_gdf = tracts_2017_gdf.to_crs(neighborhood_gdf.crs)
e = time.time()
print('Got {0:,.0f} 2017 census tracts in {1:.3f} seconds.'.format(len(tracts_2017_gdf), e-s))

s = time.time()
census_tract_to_neighborhood_filename = shapefile_dir + 'census_tract_to_neighborhood.csv'
census_tract_to_neighborhood_df = pandas.read_csv(census_tract_to_neighborhood_filename, low_memory=False)
census_tract_to_neighborhood_df['NAME10'] = pandas.to_numeric(census_tract_to_neighborhood_df['NAME10'], errors='coerce')
census_tract_to_neighborhood_df = census_tract_to_neighborhood_df.set_index('NAME10')

tract_shapes_with_csa_df = tracts_2017_gdf.join(census_tract_to_neighborhood_df, how='left')

csa_df = pandas.DataFrame(data=tract_shapes_with_csa_df.groupby('CSA2010').size(), columns=['ignore'])

csa_df = csa_df.assign(geometry='')
for thiscsa, thisrow in csa_df.iterrows():
    tracts_here = tract_shapes_with_csa_df[tract_shapes_with_csa_df['CSA2010'] == thiscsa].index.values.tolist()
    geos_here = tract_shapes_with_csa_df[tract_shapes_with_csa_df.index.isin(tracts_here)].geometry
    csa_df.loc[thiscsa, 'geometry'] = geos_here.unary_union

csa_df = csa_df.drop('ignore', axis=1)

csa_gdf = gpd.GeoDataFrame(csa_df, crs=tracts_2017_gdf.crs, geometry='geometry')
csa_gdf = csa_gdf.to_crs(neighborhood_gdf.crs)
e = time.time()

print('Assembled those census tracts into {0:,.0f} CSAs in {1:,.3f} seconds.'.format(len(csa_gdf), e-s))

s = time.time()
blockgroup_filename = shapefile_dir + 'cb_2017_24_bg_500k/cb_2017_24_bg_500k.shp'
block_group_gdf = gpd.read_file(blockgroup_filename)
block_group_gdf['STATEFP'] = pandas.to_numeric(block_group_gdf['STATEFP'], errors='coerce')
block_group_gdf['COUNTYFP'] = pandas.to_numeric(block_group_gdf['COUNTYFP'], errors='coerce')
block_group_gdf = block_group_gdf[(block_group_gdf['STATEFP'] == 24) & (block_group_gdf['COUNTYFP'] == 510)]
block_group_gdf = block_group_gdf.to_crs(neighborhood_gdf.crs)
e = time.time()

print('Read {0:,.0f} block groups in {1:.3f} seconds...'.format(len(block_group_gdf), e-s))

s = time.time()
zipcodes_filename = shapefile_dir + 'zipcodes/ZIP_Codes.shp'
zipcodes_gdf = gpd.read_file(zipcodes_filename)
zipcodes_gdf = zipcodes_gdf.set_index('ZIPCODE1')
e = time.time()
print('Read {0:,.0f} zip code shapes in {1:,.3f} seconds.'.format(len(zipcodes_gdf), e-s))


outline_filename = shapefile_dir + 'baltimore_city_polygon/baltimore_city_polygon.shp'
city_outline_gdf = gpd.read_file(outline_filename)
city_outline_gdf = city_outline_gdf.to_crs(neighborhood_gdf.crs)
print('\nRead city outline...')

water_filename = shapefile_dir + 'water/water.shp'
water_gdf = gpd.read_file(water_filename)
water_gdf = water_gdf.set_index('OBJECTID')
print('Read water outlines...')

s = time.time()
streets_filename = shapefile_dir + 'streets/streetcl.shp'
streets_gdf = gpd.read_file(streets_filename)
streets_gdf = streets_gdf.set_index('OBJECTID')
streets_gdf.to_crs(neighborhood_gdf.crs)
e = time.time()
print('Read {0:,.0f} street centerlines in {1:,.1f} seconds.'.format(len(streets_gdf), e-s))

print('\nDone!')

## Sort through one tract at a time to identify neighborhoods

In [ ]:
thistract = 2602.03
thistractstr = '260203'
nhoods = ['Cedonia']
show_streets = True
show_water = False

fig, ax = plt.subplots(figsize=(15,15))

z = tracts_2010_gdf[tracts_2010_gdf.index == thistract]
z.plot(ax=ax, color='none', edgecolor='black', linewidth=8, linestyle='-', alpha=0.75)

for ix, thisrow in z.iterrows():
    annotator = ix
    ax.annotate(annotator, xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), ha='center', va='center', fontsize=18, alpha=1)

zb = block_group_gdf[block_group_gdf['TRACTCE'] == thistractstr]
zb.plot(ax=ax, color='none', edgecolor='black', linewidth=8, linestyle=':')
for ix, thisrow in zb.iterrows():
    annotator = thisrow['BLKGRPCE']
    ax.annotate(annotator, xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), ha='center', va='center', fontsize=18, alpha=1)

zz = neighborhood_gdf[neighborhood_gdf.index.map(lambda x: x in nhoods)]
zz.plot(ax=ax, color='none', edgecolor='red', linewidth=8, linestyle='-')
for ix, thisrow in zz.iterrows():
    annotator = ix
    ax.annotate(annotator, xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), ha='center', va='center', color='red', fontsize=18, alpha=0.75)

grapharea_s = gpd.GeoSeries(data=[
    Polygon([
        (plt.xlim()[0],plt.ylim()[0]), 
        (plt.xlim()[1],plt.ylim()[0]), 
        (plt.xlim()[1],plt.ylim()[1]), 
        (plt.xlim()[0],plt.ylim()[1])
    ])
], crs=neighborhood_gdf.crs)

grapharea_gdf = gpd.GeoDataFrame(data=None, geometry=grapharea_s.geometry, crs=neighborhood_gdf.crs)

if (show_streets):
    sy_gdf = streets_gdf[
        (streets_gdf['SUBTYPE'].apply(lambda x: x in ['STRPRD']))
    ]
    sy_gdf = sy_gdf[sy_gdf.geometry.within(grapharea_gdf.loc[0].geometry)]
    sy_gdf.geometry.plot(ax=ax, linewidth=1, color='purple')
    for ix, thisrow in sy_gdf.iterrows():
        annotator = thisrow['FULLNAME']
        ax.annotate(annotator, xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), ha='center', va='center', color='purple', 
                    fontsize=8)
if (show_water):
    ww_gdf = water_gdf[water_gdf.geometry.within(grapharea_gdf.loc[0].geometry)]
    ww_gdf.geometry.plot(ax=ax, linewidth=1, color='cyan')
    for ix, thisrow in ww_gdf.iterrows():
        annotator = thisrow['NAME']
        ax.annotate(annotator, xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), ha='center', va='center', color='blue', fontsize=8)
        
#ax.tick_params(axis='both', which='both', bottom=False, left=False, labelleft=False, labelbottom=False)
grapharea_gdf.plot(ax=ax, color='none', edgecolor='orange', linewidth=6)

plt.show()

## Remove neighborhoods with zero population

In [ ]:
s = time.time()
zeropops = len(neighborhood_gdf[neighborhood_gdf['Population'] == 0])
neighborhood_gdf = neighborhood_gdf[neighborhood_gdf['Population'] > 0]
e = time.time()
print('removed {0:,.0f} zero-population neighborhoods in {1:,.3f} seconds.'.format(zeropops, e-s))

# Compare block groups, census tracts, and community statistical areas

In [ ]:
fig, ax = plt.subplots(figsize=(38,38))

block_group_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2, linestyle=':', alpha=0.5)

tracts_2017_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2, linestyle='-', alpha=0.75)

for ix, thisrow in block_group_gdf.iterrows():
    annotator = thisrow['NAME']
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=11, color='black', alpha=0.5)

for ix, thisrow in tracts_2017_gdf.iterrows():
    annotator = ix
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=18, alpha=0.75)

csa_gdf.plot(ax=ax, color='none', edgecolor='green', linewidth=4)

for ix, thisrow in csa_gdf.iterrows():
    annotator = ix.replace('-', '- ')
    annotator = annotator.replace('/', '/ ')
    annotator = annotator.replace(' ', '\n')
    annotator = annotator.upper()
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=24, color='green')

#neighborhood_gdf.plot(ax=ax, color='none', edgecolor='red', linewidth=2)

#water_gdf[water_gdf['NAME'] == 'Harbor'].plot(ax=ax, color='blue')

ax.tick_params(axis='both', which='both', bottom=False, left=False, labelleft=False, labelbottom=False)

#print('saving...')
#plt.savefig('/home/idies/workspace/Storage/raddick/persistent/cra/baltimore_boundary_guide_csa.svg', format='svg')
#print('ok')
plt.show()
#csa_gdf

# Compare block groups, census tracts, and neighborhoods

In [ ]:
fig, ax = plt.subplots(figsize=(38,38))

block_group_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2, linestyle=':', alpha=0.5)

tracts_2017_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2, linestyle='-')


for ix, thisrow in block_group_gdf.iterrows():
    annotator = thisrow['NAME']
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=9, color='black', alpha=0.5)

for ix, thisrow in tracts_2017_gdf.iterrows():
    annotator = ix
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=12)
    
csa_gdf.plot(ax=ax, color='none', edgecolor='green', linewidth=4)

#for ix, thisrow in csa_gdf.iterrows():
#    annotator = ix.replace('-', '- ')
#    annotator = annotator.replace('/', '/ ')
#    annotator = annotator.replace(' ', '\n')
#    annotator = annotator.upper()
#    ax.annotate(annotator, 
#                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
#                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
#                ha='center', va='center', fontsize=24, color='green')

neighborhood_gdf.plot(ax=ax, color='none', edgecolor='red', linewidth=2)

for ix, thisrow in neighborhood_gdf.iterrows():
    annotator = ix.replace('-', '- ')
    annotator = annotator.replace('/', '/ ')
    annotator = annotator.replace(' ', '\n')
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=14, color='red')

#water_gdf.plot(ax=ax, color='blue')
water_gdf[water_gdf['NAME'] == 'Harbor'].plot(ax=ax, color='blue')

ax.tick_params(axis='both', which='both', bottom=False, left=False, labelleft=False, labelbottom=False)

plt.show()
#csa_gdf
#plt.savefig('/home/idies/workspace/Storage/raddick/persistent/cra/baltimore_boundary_guide.svg', format='svg')
print('ok')

In [ ]:
cols = neighborhood_gdf.columns.tolist()
cols.remove('created_us')
cols.remove('created_da')
cols.remove('last_edite')
cols.remove('last_edi_1')
cols.remove('ShapeSTAre')
cols.remove('ShapeSTLen')
cols.remove('geometry')
#os.getcwd()
neighborhood_gdf[cols].to_csv('neighborhood_full_data.csv', encoding='utf-8')
print('ok')

In [ ]:
fig, ax = plt.subplots(figsize=(38,38))

block_group_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2, linestyle=':', alpha=0.5)
tracts_2017_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2, linestyle='-', alpha=0.75)

for ix, thisrow in block_group_gdf.iterrows():
    annotator = thisrow['NAME']
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=11, color='black', alpha=0.5)

for ix, thisrow in tracts_2017_gdf.iterrows():
    annotator = ix
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=18, alpha=0.75)

neighborhood_gdf.plot(ax=ax, color='none', edgecolor='red', linewidth=2)

ax.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')
plt.show()

# Compare zip codes and neighborhoods

In [ ]:
fig, ax = plt.subplots(figsize=(38,38))

neighborhood_gdf.plot(ax=ax, color='none', edgecolor='red', linewidth=2)

for ix, thisrow in neighborhood_gdf.iterrows():
    annotator = ix.replace('-', '- ')
    annotator = annotator.replace('/', '/ ')
    annotator = annotator.replace(' ', '\n')
#    print(thisrow.geometry.centroid.x)
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=14, color='red')

zipcodes_gdf.plot(ax=ax, color='none', edgecolor='blue', linewidth=5)
#print('------------------------\n\n')
for ix, thisrow in zipcodes_gdf.iterrows():
    annotator = ix
#    print(annotator)
#    print(thisrow.geometry.centroid.x)
    ax.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', fontsize=36, color='blue')

ax.tick_params(axis='both', which='both', bottom=False, left=False, labelleft=False, labelbottom=False)
plt.show()

In [ ]:
neighborhood_gdf[neighborhood_gdf.index.map(lambda x: x[0] == 'W')]
#block_group_gdf[block_group_gdf['TRACTCE'] == '010400']
#print('ok')

In [ ]:
#streets_gdf.sample(2).T
streets_gdf.groupby('SUBTYPE').size()

In [ ]:
# STREX: expresswway
# STRR: ramp
# STRTN: tunnel
# STRPRD: primary road
# STRALY: alley
fig, ax = plt.subplots(figsize=(15,15))
streets_gdf[streets_gdf['SUBTYPE'].apply(lambda x: x in ['STREX'])].plot(ax=ax, color='red')
streets_gdf[streets_gdf['SUBTYPE'].apply(lambda x: x in ['STRPRD'])].plot(ax=ax, color='blue')
plt.show()

In [ ]:
block_group_gdf[block_group_gdf['TRACTCE'] == '250500']